In [2]:
import lightgbm as lgb
from sklearn import metrics

import pandas as pd

In [4]:
!ls -lah

total 889568
drwxr-xr-x  12 avkash  staff   384B Mar 27 08:36 .
drwxr-xr-x  13 avkash  staff   416B Mar 26 22:49 ..
drwxr-xr-x   6 avkash  staff   192B Mar 27 08:22 .ipynb_checkpoints
-rw-r--r--   1 avkash  staff    15K Mar 27 08:30 Untitled.ipynb
-rw-r--r--   1 avkash  staff    54M Mar 27 08:35 ca_daily_fire_2000_2021.csv
-rw-r--r--   1 avkash  staff   3.8M Mar 27 08:36 ca_fire_test.csv
-rw-r--r--   1 avkash  staff    60M Mar 27 08:36 ca_fire_train.csv
-rw-r--r--   1 avkash  staff   7.5M Mar 27 08:36 ca_fire_valid.csv
-rw-r--r--   1 avkash  staff   3.8M Mar 27 08:36 feature-engineering-ca-wildfire-data.ipynb
-rw-r--r--   1 avkash  staff    31K Mar 27 08:14 ml-ca-wildfire-ml-ready-data.ipynb
-rw-r--r--   1 avkash  staff   289M Mar 27 08:29 usa_daily_fire_2000_2021.csv
-rw-r--r--   1 avkash  staff   312K Mar 27 08:36 wildfire_data_creation.ipynb


In [3]:
train = pd.read_csv('ca_fire_train.csv')
valid = pd.read_csv('ca_fire_valid.csv')
test = pd.read_csv('ca_fire_test.csv')

In [4]:
train.shape

(899396, 12)

In [5]:
valid.shape

(104784, 12)

In [6]:
test.shape

(0, 12)

In [7]:
train.groupby('year').count()

,latitude,longitude,month,fire_count,fire,fire_cnt_before,fire_before,fire_cnt_last_year,fire_last_year,fire_cnt_last_year_same_month,fire_last_year_same_month
year,,,,,,,,,,,
2001,8732,8732,8732,8732,8732,8732,8732,8732,8732,8732,8732
2002,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392
2003,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392
2004,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392
2005,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392
2006,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392
2007,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392
2008,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392
2009,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392


In [8]:
valid.groupby('year').count()

,latitude,longitude,month,fire_count,fire,fire_cnt_before,fire_before,fire_cnt_last_year,fire_last_year,fire_cnt_last_year_same_month,fire_last_year_same_month
year,,,,,,,,,,,
2019,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392
2020,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392,52392


In [9]:
test.groupby('year').count()

,latitude,longitude,month,fire_count,fire,fire_cnt_before,fire_before,fire_cnt_last_year,fire_last_year,fire_cnt_last_year_same_month,fire_last_year_same_month
year,,,,,,,,,,,


In [10]:
print(train.shape)
print(valid.shape)
print(test.shape)

(899396, 12)
(104784, 12)
(0, 12)


In [11]:
train.head()

,latitude,longitude,year,month,fire_count,fire,fire_cnt_before,fire_before,fire_cnt_last_year,fire_last_year,fire_cnt_last_year_same_month,fire_last_year_same_month
0,32.5,-117.0,2003,1,1.0,1,0.083333,0.083333,0.000000,0.000000,0.0,0.0
1,32.5,-117.0,2005,8,0.0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,32.5,-117.0,2017,8,0.0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,32.5,-117.0,2002,8,0.0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,32.5,-117.0,2004,8,0.0,0,0.000000,0.000000,0.083333,0.083333,0.0,0.0


In [12]:
features = [
    'latitude', 'longitude', 'month',
    'fire_cnt_before', 'fire_before',
    'fire_cnt_last_year', 'fire_last_year',
    'fire_cnt_last_year_same_month', 'fire_last_year_same_month'
]

In [13]:
train.columns

Index(['latitude', 'longitude', 'year', 'month', 'fire_count', 'fire',
       'fire_cnt_before', 'fire_before', 'fire_cnt_last_year',
       'fire_last_year', 'fire_cnt_last_year_same_month',
       'fire_last_year_same_month'],
      dtype='object')

In [14]:
train_data = lgb.Dataset(train[features], label=train.fire)
valid_data = lgb.Dataset(valid[features], label=valid.fire)


In [15]:
parameters = {'num_leaves': 10, 'max_depth': 8, 'objective': 'binary', 'metric': 'auc'}
num_round = 500

In [3]:
model = lgb.train(parameters, train_data, num_round, valid_sets=[valid_data],
                  early_stopping_rounds=5, verbose_eval=50)

NameError: name 'parameters' is not defined